<a href="https://colab.research.google.com/github/Abubakar919666/Python/blob/main/Customer_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
reviews = pd.read_excel('customer_reviews.xlsx')

In [22]:
reviews.sample(10)

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
199,200,30,17,2025-01-09,3,"Good quality, but could be cheaper."
415,416,15,17,2023-04-19,5,The quality is top-notch.
471,472,69,8,2024-12-05,4,Shipping was fast and the item was well...
820,821,98,12,2025-10-19,3,"The product is okay, but the instruction..."
943,944,72,2,2024-08-11,4,The quality is top-notch.
1019,1020,100,5,2025-09-02,3,"Average experience, nothing special."
1218,1219,49,7,2025-05-28,5,"Great purchase, very satisfied."
634,635,85,16,2023-02-22,3,"The product is okay, but the instruction..."
1221,1222,57,6,2025-05-23,3,"I love this product, will buy again!"
216,217,62,11,2023-08-19,5,Amazing value for the price.


In [25]:
reviews['ReviewText'] = reviews['ReviewText'].str.replace(" ","")
reviews["ReviewText"]

,ReviewText
0,"Averageexperience,nothingspecial."
1,Thequalityistop-notch.
2,Fivestarsforthequickdelivery.
3,"Goodquality,butcouldbecheaper."
4,"Averageexperience,nothingspecial."
...,...
1358,Notworththemoney.
1359,"Averageexperience,nothingspecial."
1360,Customersupportwasveryhelpful.
1361,Productdidnotmeetmyexpectations.


In [26]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [27]:
# Download the VADER lexicon for sentiment analysis if not already present.
nltk.download('vader_lexicon')
# The VADER lexicon is a list of words and phrases with their corresponding sentiment
# ratings that's used in the VADER (Valence Aware Dictionary and sEntiment Reasoner) sentiment analysis tool.

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [28]:
sia = SentimentIntensityAnalyzer()

### A polarity score is a numerical rating that measures the overall tone of a text, phrase, or word, and indicates whether it is positive, negative, or neutral. Polarity scores are used in sentiment analysis, which is the process of analyzing opinions
### Returns the compound score, which is a normalized score between -1 (most negative) and 1 (most positive)

In [29]:
# Define a function to calculate sentiment scores using VADER
def calculate_sentiment(review):
    # Get the sentiment scores for the review text
    sentiment = sia.polarity_scores(review)
    # Return the compound score, which is a normalized score between -1 (most negative) and 1 (most positive)
    # print(sentiment)
    return sentiment['compound']
    # return sentiment

In [30]:
# test code
review = "I really really really like this product.This product is such goose bump!"

# Calculate sentiment
print(calculate_sentiment(review))  # Outputs a compound score, e.g., 0.84

0.5613


In [31]:
# Example test code
texts = [
    "Such a great invention. I am in love this product!",     # Positive
    "This product is okay.",    # Neutral
    "I hate this product."      # Negative
]

# Analyze and print the compound scores
for text in texts:
    scores = sia.polarity_scores(text)
    print(f"Text: {text}\nCompound Score: {scores['compound']}\n")


Text: Such a great invention. I am in love this product!
Compound Score: 0.8622

Text: This product is okay.
Compound Score: 0.2263

Text: I hate this product.
Compound Score: -0.5719



In [32]:
review_text = ReviewText['ReviewText']

In [33]:
# Apply sentiment analysis to calculate sentiment scores for each review
reviews['SentimentScore'] = reviews['ReviewText'].apply(calculate_sentiment)
reviews

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore
0,1,77,18,2023-12-23,3,"Averageexperience,nothingspecial.",0.0
1,2,80,19,2024-12-25,5,Thequalityistop-notch.,0.0
2,3,50,13,2025-01-26,4,Fivestarsforthequickdelivery.,0.0
3,4,78,15,2025-04-21,3,"Goodquality,butcouldbecheaper.",0.0
4,5,64,2,2023-07-16,3,"Averageexperience,nothingspecial.",0.0
...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Notworththemoney.,0.0
1359,1360,58,12,2023-11-13,2,"Averageexperience,nothingspecial.",0.0
1360,1361,96,15,2023-03-07,5,Customersupportwasveryhelpful.,0.0
1361,1362,99,2,2025-12-03,1,Productdidnotmeetmyexpectations.,0.0


In [34]:
# Define a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    # Use both the text sentiment score and the numerical rating to determine sentiment category
    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  # High rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive'  # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'  # Low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment


In [35]:
# Apply sentiment categorization using both text and rating
reviews['SentimentCategory'] = reviews.apply(lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)
reviews

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory
0,1,77,18,2023-12-23,3,"Averageexperience,nothingspecial.",0.0,Neutral
1,2,80,19,2024-12-25,5,Thequalityistop-notch.,0.0,Positive
2,3,50,13,2025-01-26,4,Fivestarsforthequickdelivery.,0.0,Positive
3,4,78,15,2025-04-21,3,"Goodquality,butcouldbecheaper.",0.0,Neutral
4,5,64,2,2023-07-16,3,"Averageexperience,nothingspecial.",0.0,Neutral
...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Notworththemoney.,0.0,Neutral
1359,1360,58,12,2023-11-13,2,"Averageexperience,nothingspecial.",0.0,Negative
1360,1361,96,15,2023-03-07,5,Customersupportwasveryhelpful.,0.0,Positive
1361,1362,99,2,2025-12-03,1,Productdidnotmeetmyexpectations.,0.0,Negative


In [36]:
# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

In [37]:
# Apply sentiment bucketing to categorize scores into defined ranges
reviews['SentimentBucket'] = reviews['SentimentScore'].apply(sentiment_bucket)
reviews

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Averageexperience,nothingspecial.",0.0,Neutral,0.0 to 0.49
1,2,80,19,2024-12-25,5,Thequalityistop-notch.,0.0,Positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Fivestarsforthequickdelivery.,0.0,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Goodquality,butcouldbecheaper.",0.0,Neutral,0.0 to 0.49
4,5,64,2,2023-07-16,3,"Averageexperience,nothingspecial.",0.0,Neutral,0.0 to 0.49
...,...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Notworththemoney.,0.0,Neutral,0.0 to 0.49
1359,1360,58,12,2023-11-13,2,"Averageexperience,nothingspecial.",0.0,Negative,0.0 to 0.49
1360,1361,96,15,2023-03-07,5,Customersupportwasveryhelpful.,0.0,Positive,0.0 to 0.49
1361,1362,99,2,2025-12-03,1,Productdidnotmeetmyexpectations.,0.0,Negative,0.0 to 0.49


In [38]:
reviews.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)

In [39]:
reviews.SentimentCategory.value_counts()

,count
SentimentCategory,
Positive,840
Neutral,290
Negative,233
